In [5]:
import os, sys
import numpy as np
import healpy as hp
import pandas as pd
import h5py
import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

os.system('module load texlive/2022')

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Times",
    "font.size"  : 13
})

matplotlib.rcParams.update({'font.size': 15})

matplotlib.rcParams['mathtext.fontset'] = 'stix'
plt.rc('text.latex', preamble=r'\usepackage{/global/cfs/cdirs/lsst/groups/WL/users/yomori/repo/nulltests_txpipe/apjfonts}')
        
#plt.rcParams["font.family"] = "Times New Roman"

plt.rcParams['axes.facecolor']   = 'w'
plt.rcParams['figure.facecolor'] = 'w'


kidscol = '#9CC363'    # light olive
descol  = 'chocolate'  # dark orange
hsccol  = '#5A8BAF'    # lightpurple

Lmod has detected the following error: The following module(s) are unknown:
"texlive/2022"

Please check the spelling or version number. Also try "module spider ..."
It is also possible your cache file is out-of-date; it may help to try:
  $ module --ignore_cache load "texlive/2022"

Also make sure that all modulefiles written in TCL start with the string
#%Module





In [6]:
from scipy.optimize import curve_fit
import numpy as np
import warnings

def fit_straight_line(x, y, y_err=None):
    """
    Use scipy to fit a straight line, with errors bars in y.

    Parameters
    ----------
    x: array
        x-coordinate
    y: array
        y-coordinate
    y_err: array/float
        optional, default=None, errors are 1D std. dev.


    Returns
    -------
    m: float
        gradient

    c: float
        intercept
    """
    if x.size == 0:
        print("ERROR: No data for straight line fit. Returning m=0 c=0")
        return 0.0, 0.0, np.array([[1.0, 0.0], [0.0, 1.0]])
    
    if x.size != y.size:
        raise ValueError("x and y must have the same length")

    try:
        popt, cov = curve_fit(line, x, y, sigma=y_err)
    except RuntimeError:
        print("ERROR: Straight line fit failed. Returning m=0 c=0")
        return 0.0, 0.0, np.array([[1.0, 0.0], [0.0, 1.0]])
    m = popt[0]
    c = popt[1]
    return m,c, cov

def line(slp,vrbl,intcpt):
    return slp * vrbl + intcpt

def calc_chi2(y, err, yfit , v = False):
    """
    Compute chi2 between data and fitted curve

    Parameters
    ----------
    y: array
        y values of data
    err: array (1D or 2D)
        either error bars (if independent data points)
        or covariance matrix
    yfit: array
        fitted values of data
    v: bool
        verbose output

    Returns
    -------
    chi2: float
    """
    if err.shape == (len(y),len(y)):
        #use full covariance
        if v:
            print('cov_mat chi2')
        inv_cov = np.linalg.inv( np.matrix(err) )
        chi2 = 0
        for i in range(len(y)):
            for j in range(len(y)):
                chi2 = chi2 + (y[i]-yfit[i])*inv_cov[i,j]*(y[j]-yfit[j])
        return chi2
        
    elif err.shape == (len(y),):
        if v:
            print('diagonal chi2')
        return sum(((y-yfit)**2.)/(err**2.))
    else:
        raise IOError('error in err or cov_mat input shape')
    


In [7]:
import pickle
with open("meanshear.pkl",'rb') as file:
    dic = pickle.load(file)

In [8]:
idx = np.where(np.isfinite(dic['KiDS-1000']['g_psfg']['mu1']))[0]
        
slope11, intercept11, mc_cov11 = fit_straight_line(dic['KiDS-1000']['g_psfg']['mu1'][idx], dic['KiDS-1000']['g_psfg']['meang1_psfg1'][idx]-np.mean(dic['KiDS-1000']['g_psfg']['meang1_psfg1'][idx]), dic['KiDS-1000']['g_psfg']['stdg1_psfg1'][idx])
slope12, intercept12, mc_cov12 = fit_straight_line(dic['KiDS-1000']['g_psfg']['mu2'][idx], dic['KiDS-1000']['g_psfg']['meang1_psfg2'][idx]-np.mean(dic['KiDS-1000']['g_psfg']['meang1_psfg2'][idx]), dic['KiDS-1000']['g_psfg']['stdg1_psfg2'][idx])
slope21, intercept21, mc_cov21 = fit_straight_line(dic['KiDS-1000']['g_psfg']['mu1'][idx], dic['KiDS-1000']['g_psfg']['meang2_psfg1'][idx]-np.mean(dic['KiDS-1000']['g_psfg']['meang2_psfg1'][idx]), dic['KiDS-1000']['g_psfg']['stdg2_psfg1'][idx])
slope22, intercept22, mc_cov22 = fit_straight_line(dic['KiDS-1000']['g_psfg']['mu2'][idx], dic['KiDS-1000']['g_psfg']['meang2_psfg2'][idx]-np.mean(dic['KiDS-1000']['g_psfg']['meang2_psfg2'][idx]), dic['KiDS-1000']['g_psfg']['stdg2_psfg2'][idx])

print(slope11, np.sqrt(np.diag(mc_cov11)[0]))
print(slope12, np.sqrt(np.diag(mc_cov12)[0]))
print(slope21, np.sqrt(np.diag(mc_cov21)[0]))
print(slope22, np.sqrt(np.diag(mc_cov22)[0]))

0.00980770762773485 0.003588493942531237
0.005716697535565843 0.003620480126323612
-0.0024935185832704455 0.004041713576354446
0.006622368390388903 0.006770671479971271


In [9]:
idx = np.where(np.isfinite(dic['KiDS-1000']['g_psfT']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['KiDS-1000']['g_psfT']['mu'][idx], dic['KiDS-1000']['g_psfT']['meang1'][idx]-np.mean(dic['KiDS-1000']['g_psfT']['meang1'][idx]), dic['KiDS-1000']['g_psfT']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['KiDS-1000']['g_psfT']['mu'][idx], dic['KiDS-1000']['g_psfT']['meang2'][idx]-np.mean(dic['KiDS-1000']['g_psfT']['meang2'][idx]), dic['KiDS-1000']['g_psfT']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

-0.00758397932591841 0.0023839883994455358
0.0036138100516336096 0.0016496594432807757


In [10]:
idx = np.where(np.isfinite(dic['KiDS-1000']['g_T']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['KiDS-1000']['g_T']['mu'][idx], dic['KiDS-1000']['g_T']['meang1'][idx]-np.mean(dic['KiDS-1000']['g_T']['meang1'][idx]), dic['KiDS-1000']['g_T']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['KiDS-1000']['g_T']['mu'][idx], dic['KiDS-1000']['g_T']['meang2'][idx]-np.mean(dic['KiDS-1000']['g_T']['meang2'][idx]), dic['KiDS-1000']['g_T']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

-0.003257755466639441 0.00038672731882865224
-8.304928377539656e-05 0.0004133477450395128


In [12]:
idx = np.where(np.isfinite(dic['KiDS-1000']['g_snr']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['KiDS-1000']['g_snr']['mu'][idx], dic['KiDS-1000']['g_snr']['meang1'][idx]-np.mean(dic['KiDS-1000']['g_snr']['meang1'][idx]), dic['KiDS-1000']['g_snr']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['KiDS-1000']['g_snr']['mu'][idx], dic['KiDS-1000']['g_snr']['meang2'][idx]-np.mean(dic['KiDS-1000']['g_snr']['meang2'][idx]), dic['KiDS-1000']['g_snr']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

1.0782017765610275e-05 7.446081310820361e-06
-1.3329141412286788e-05 5.708239171226594e-06


In [ ]:
#################################################

In [13]:
idx = np.where(np.isfinite(dic['DES-Y3']['g_psfg']['mu1']))[0]
        
slope11, intercept11, mc_cov11 = fit_straight_line(dic['DES-Y3']['g_psfg']['mu1'][idx], dic['DES-Y3']['g_psfg']['meang1_psfg1'][idx]-np.mean(dic['DES-Y3']['g_psfg']['meang1_psfg1'][idx]), dic['DES-Y3']['g_psfg']['stdg1_psfg1'][idx])
slope12, intercept12, mc_cov12 = fit_straight_line(dic['DES-Y3']['g_psfg']['mu2'][idx], dic['DES-Y3']['g_psfg']['meang1_psfg2'][idx]-np.mean(dic['DES-Y3']['g_psfg']['meang1_psfg2'][idx]), dic['DES-Y3']['g_psfg']['stdg1_psfg2'][idx])
slope21, intercept21, mc_cov21 = fit_straight_line(dic['DES-Y3']['g_psfg']['mu1'][idx], dic['DES-Y3']['g_psfg']['meang2_psfg1'][idx]-np.mean(dic['DES-Y3']['g_psfg']['meang2_psfg1'][idx]), dic['DES-Y3']['g_psfg']['stdg2_psfg1'][idx])
slope22, intercept22, mc_cov22 = fit_straight_line(dic['DES-Y3']['g_psfg']['mu2'][idx], dic['DES-Y3']['g_psfg']['meang2_psfg2'][idx]-np.mean(dic['DES-Y3']['g_psfg']['meang2_psfg2'][idx]), dic['DES-Y3']['g_psfg']['stdg2_psfg2'][idx])

print(slope11, np.sqrt(np.diag(mc_cov11)[0]))
print(slope12, np.sqrt(np.diag(mc_cov12)[0]))
print(slope21, np.sqrt(np.diag(mc_cov21)[0]))
print(slope22, np.sqrt(np.diag(mc_cov22)[0]))

3.7970726958254455e-05 0.0030069361199564324
-0.002421975120217709 0.0020013977843730732
-0.0046549544317699 0.002859669952127049
-0.0036280017946138253 0.0028005655392911617


In [9]:
idx = np.where(np.isfinite(dic['DES-Y3']['g_psfT']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['DES-Y3']['g_psfT']['mu'][idx], dic['DES-Y3']['g_psfT']['meang1'][idx]-np.mean(dic['DES-Y3']['g_psfT']['meang1'][idx]), dic['DES-Y3']['g_psfT']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['DES-Y3']['g_psfT']['mu'][idx], dic['DES-Y3']['g_psfT']['meang2'][idx]-np.mean(dic['DES-Y3']['g_psfT']['meang2'][idx]), dic['DES-Y3']['g_psfT']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

0.00040635228267147023 0.0002477457869727419
7.745841401753567e-05 0.0004264537259936164


In [10]:
idx = np.where(np.isfinite(dic['DES-Y3']['g_T']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['DES-Y3']['g_T']['mu'][idx], dic['DES-Y3']['g_T']['meang1'][idx]-np.mean(dic['DES-Y3']['g_T']['meang1'][idx]), dic['DES-Y3']['g_T']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['DES-Y3']['g_T']['mu'][idx], dic['DES-Y3']['g_T']['meang2'][idx]-np.mean(dic['DES-Y3']['g_T']['meang2'][idx]), dic['DES-Y3']['g_T']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

-0.001028458911876278 0.00018134676671109187
9.49478598752113e-05 0.00011997884801148157


In [11]:
idx = np.where(np.isfinite(dic['DES-Y3']['g_snr']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['DES-Y3']['g_snr']['mu'][idx], dic['DES-Y3']['g_snr']['meang1'][idx]-np.mean(dic['DES-Y3']['g_snr']['meang1'][idx]), dic['DES-Y3']['g_snr']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['DES-Y3']['g_snr']['mu'][idx], dic['DES-Y3']['g_snr']['meang2'][idx]-np.mean(dic['DES-Y3']['g_snr']['meang2'][idx]), dic['DES-Y3']['g_snr']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

-2.0208688691144363e-06 7.241457880313813e-07
4.6288462058821055e-07 5.969537655348139e-07


In [ ]:
#####################################################

In [14]:
idx = np.where(np.isfinite(dic['HSC-Y3']['g_psfg']['mu1']))[0]
        
slope11, intercept11, mc_cov11 = fit_straight_line(dic['HSC-Y3']['g_psfg']['mu1'][idx], dic['HSC-Y3']['g_psfg']['meang1_psfg1'][idx]-np.mean(dic['HSC-Y3']['g_psfg']['meang1_psfg1'][idx]), dic['HSC-Y3']['g_psfg']['stdg1_psfg1'][idx])
slope12, intercept12, mc_cov12 = fit_straight_line(dic['HSC-Y3']['g_psfg']['mu2'][idx], dic['HSC-Y3']['g_psfg']['meang1_psfg2'][idx]-np.mean(dic['HSC-Y3']['g_psfg']['meang1_psfg2'][idx]), dic['HSC-Y3']['g_psfg']['stdg1_psfg2'][idx])
slope21, intercept21, mc_cov21 = fit_straight_line(dic['HSC-Y3']['g_psfg']['mu1'][idx], dic['HSC-Y3']['g_psfg']['meang2_psfg1'][idx]-np.mean(dic['HSC-Y3']['g_psfg']['meang2_psfg1'][idx]), dic['HSC-Y3']['g_psfg']['stdg2_psfg1'][idx])
slope22, intercept22, mc_cov22 = fit_straight_line(dic['HSC-Y3']['g_psfg']['mu2'][idx], dic['HSC-Y3']['g_psfg']['meang2_psfg2'][idx]-np.mean(dic['HSC-Y3']['g_psfg']['meang2_psfg2'][idx]), dic['HSC-Y3']['g_psfg']['stdg2_psfg2'][idx])

print(slope11, np.sqrt(np.diag(mc_cov11)[0]))
print(slope12, np.sqrt(np.diag(mc_cov12)[0]))
print(slope21, np.sqrt(np.diag(mc_cov21)[0]))
print(slope22, np.sqrt(np.diag(mc_cov22)[0]))

0.0025589386555993363 0.0038605700064791785
-0.0019495062338309057 0.003515536540160969
0.0022363440669734705 0.00397629759428652
0.01677240444138269 0.0033036569415903147


In [15]:
idx = np.where(np.isfinite(dic['HSC-Y3']['g_psfT']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['HSC-Y3']['g_psfT']['mu'][idx], dic['HSC-Y3']['g_psfT']['meang1'][idx]-np.mean(dic['HSC-Y3']['g_psfT']['meang1'][idx]), dic['HSC-Y3']['g_psfT']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['HSC-Y3']['g_psfT']['mu'][idx], dic['HSC-Y3']['g_psfT']['meang2'][idx]-np.mean(dic['HSC-Y3']['g_psfT']['meang2'][idx]), dic['HSC-Y3']['g_psfT']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

0.004117116068562921 0.002485058824227244
-0.00021560661947545793 0.0020538172691511526


In [14]:
idx = np.where(np.isfinite(dic['HSC-Y3']['g_T']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['HSC-Y3']['g_T']['mu'][idx], dic['HSC-Y3']['g_T']['meang1'][idx]-np.mean(dic['HSC-Y3']['g_T']['meang1'][idx]), dic['HSC-Y3']['g_T']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['HSC-Y3']['g_T']['mu'][idx], dic['HSC-Y3']['g_T']['meang2'][idx]-np.mean(dic['HSC-Y3']['g_T']['meang2'][idx]), dic['HSC-Y3']['g_T']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

0.002144107608190472 0.00040116786312237253
-0.00023839664432874277 0.000331485293330499


In [15]:
idx = np.where(np.isfinite(dic['HSC-Y3']['g_snr']['mu']))[0]

slope1, intercept1, mc_cov1 = fit_straight_line(dic['HSC-Y3']['g_snr']['mu'][idx], dic['HSC-Y3']['g_snr']['meang1'][idx]-np.mean(dic['HSC-Y3']['g_snr']['meang1'][idx]), dic['HSC-Y3']['g_snr']['stdg1'][idx])
slope2, intercept2, mc_cov2 = fit_straight_line(dic['HSC-Y3']['g_snr']['mu'][idx], dic['HSC-Y3']['g_snr']['meang2'][idx]-np.mean(dic['HSC-Y3']['g_snr']['meang2'][idx]), dic['HSC-Y3']['g_snr']['stdg2'][idx])

print(slope1,np.sqrt(np.diag(mc_cov1)[0]))
print(slope2,np.sqrt(np.diag(mc_cov2)[0]))

-1.406905586600798e-06 1.583435233070325e-06
-1.7426583151384545e-06 1.2941899862738107e-06
